### A5 E1. using data frame

In [1]:
import pandas as pd
import numpy as np
from scipy import interpolate

# Parameters
rho = 1.225  # density [kg/m^3]
D = 82  # rotor_diameter [m]
T = 8760  # total hours/years [h]
A = np.pi * (D / 2) ** 2 # swept area [m^2]

# Processing power curve data
power_curve_df = pd.read_csv('Senvion_MM82.csv')
cut_in_ws = power_curve_df[power_curve_df['P'] > 0]['ws'].min()
cut_out_ws = power_curve_df[power_curve_df['P'] > 0]['ws'].max()
cp_i = interpolate.interp1d(power_curve_df['ws'], power_curve_df['cp']) #Interpolate cp

# Processing timeseries data
timeseries_df = pd.read_csv('timeseries_la_haute_borne_2017.csv')
timeseries_df.fillna(0, inplace=True) # change NaN values to zero. 'Inplace=True' => modify the data frame directly and'Inplace=False' will return a new data frame.
timeseries_df['WS'] = np.sqrt(timeseries_df['U']**2 + timeseries_df['V']**2)

def calculate_APP_entire_timeseries(timeseries_df, cp_i):
    
    valid_ws = timeseries_df['WS'].between(cut_in_ws, cut_out_ws, inclusive='both')
    cp = cp_i(timeseries_df['WS'][valid_ws])
    delta_t = 10 / 60  # 10 min in 1h
    power = 0.5 * rho * A * cp * timeseries_df['WS'][valid_ws]**3 * delta_t
    APP = power.sum() / T

    print(f"The APP based on the entired timeseries is: {APP/1000:.2f} kW")

    return APP

#Execution
APP_ET = calculate_APP_entire_timeseries(timeseries_df, cp_i)


The APP based on the entired timeseries is: 314.67 kW


### A5 E1. using array

In [2]:
import numpy as np
from scipy import interpolate

# Parameters
rho = 1.225  # density [kg/m^3]
D = 82  # rotor_diameter [m]
A = np.pi * (D / 2) ** 2  # swept area [m^2]
T = 8760  # total hours/years [h]

# Processing power curve data
power_curve_data = np.genfromtxt('Senvion_MM82.csv', delimiter=',', skip_header=1)
ws_power_curve = power_curve_data[:, 0]
cp_power_curve = power_curve_data[:, 3]
cut_in_ws = ws_power_curve[cp_power_curve > 0].min()
cut_out_ws = ws_power_curve[cp_power_curve > 0].max()

cp_i = interpolate.interp1d(ws_power_curve, cp_power_curve) #Interpolate cp

# Processing timeseries data
timeseries_data = np.genfromtxt('timeseries_la_haute_borne_2017.csv', delimiter=',', skip_header=1)
U_timeseries = np.nan_to_num(timeseries_data[:, 1])  # Replace NaNs with 0
V_timeseries = np.nan_to_num(timeseries_data[:, 2]) 
ws_timeseries = np.sqrt(U_timeseries**2 + V_timeseries**2)

# Function to calculate APP using arrays
def calculate_APP_entire_timeseries(ws, cp_i):
    valid_ws_range = (ws >= cut_in_ws) & (ws <= cut_out_ws)
    valid_ws = ws[valid_ws_range]
    cp = cp_i(valid_ws)
    delta_t = 10 / 60  # 10 min in 1h
    power = 0.5 * rho * A * cp * valid_ws ** 3 * delta_t
    APP = np.sum(power) / T

    print(f"The APP based on the entire timeseries is: {APP/1000:.2f} kW")

    return APP

# Calculate APP
APP_ET_array = calculate_APP_entire_timeseries(ws_timeseries, cp_i)

The APP based on the entire timeseries is: 314.67 kW


In [3]:
import numpy as np
from scipy import interpolate

# Parameters
rho = 1.225  # Air density [kg/m^3]
D = 82  # Rotor diameter [m]
A = np.pi * (D / 2) ** 2  # Swept area [m^2]

# Processing power curve data
power_curve_data = np.genfromtxt('Senvion_MM82.csv', delimiter=',', skip_header=1)
ws_power_curve = power_curve_data[:, 0]
cp_power_curve = power_curve_data[:, 3]
cut_in_ws = ws_power_curve[cp_power_curve > 0].min()
cut_out_ws = ws_power_curve[cp_power_curve > 0].max()

cp_i = interpolate.interp1d(ws_power_curve, cp_power_curve) # interpolate cp

# Processing timeseries data
timeseries_data = np.genfromtxt('timeseries_la_haute_borne_2017.csv', delimiter=',', skip_header=1)
U_timeseries = np.nan_to_num(timeseries_data[:, 1])  # Replace NaNs with 0
V_timeseries = np.nan_to_num(timeseries_data[:, 2]) 
ws_timeseries = np.sqrt(U_timeseries**2 + V_timeseries**2)

def calculate_APP_average_ws(ws, cp_i):
    valid_ws_range = (ws >= cut_in_ws) & (ws <= cut_out_ws)
    valid_ws = ws[valid_ws_range]
    average_ws = np.mean(valid_ws)
    cp = cp_i(average_ws)
    APP = 0.5 * rho * A * cp * average_ws ** 3 

    print(f"The APP based on the average windspeed is: {APP/1000:.2f} kW")

    return APP

APP_avg_ws = calculate_APP_average_ws(ws_timeseries, cp_i)

The APP based on the average windspeed is: 373.96 kW


### A6 E4.
- (a) Load the data file timeseries_la_haute_borne_2017.csv provided in exercise A5 with pandas. You will have to convert the index with pd.datetime(utc=True) to get a consistent datetime index.
- (b) Calculate the wind direction and wind speed and add them in two additional columns
- (c) How many NaN-values exist per column? In which months do the most NaN values occur?
- (d) Fill the NaN-values of the wind speed with the values of the previous time step. By how many percentage points does the mean wind speed change?

In [4]:
import pandas as pd

## Task (a)
# Load the data
timeseries_df = pd.read_csv('timeseries_la_haute_borne_2017.csv')
timeseries_df['Time'] = pd.to_datetime(timeseries_df['Time'], utc=True) #convert the 'time' column of the dataframe into datetime objects.
timeseries_df.set_index('Time', inplace=True) #designate a specific column as index

## Task (b)
# Calculate wind direction and wind speed
timeseries_df['Wind Speed'] = np.sqrt(timeseries_df['U']**2 + timeseries_df['V']**2)
timeseries_df['Wind Direction'] = (np.arctan2(timeseries_df['V'], timeseries_df['U'])* (180/np.pi) + 360) %360 # +360 to ensure all angles are positive, %360 (modulo operation) to bring all angles in the range of 0-359°

## Task (c)
# Count NaN values per column
nan_per_column = timeseries_df.isnull().sum() #Since sum() calculates as True=1 and False=0, you can count the number of NaN in each row and column by calling sum() on the result of isnull().
for column in nan_per_column.index:
    print(f"The number of NaN values in column {column} is {nan_per_column[column]}")
print()

# Count NaN values per month
nan_per_month = timeseries_df.isnull().resample('M').sum()
month_with_most_nans = nan_per_month.idxmax() # find the index of the maximum value for each column

for column in month_with_most_nans.index:
    month_name = month_with_most_nans[column].strftime('%B') #format the datetime object into a string representing the month's name
    print(f"The month with the most NaN values in column {column} is {month_name}")
print()

## Task (d)   
orig_mean_ws = timeseries_df['Wind Speed'].mean()
timeseries_df['Wind Speed'].fillna(method='ffill',inplace=True) # fill with the last valid value before NaN, modify the original df
new_mean_ws = timeseries_df['Wind Speed'].mean()

change_percentage = ((new_mean_ws - orig_mean_ws) / orig_mean_ws) * 100

print(f"the orginal mean wind speed was: {orig_mean_ws:.3f} m/s")
print(f"the new mean wind speed is: {new_mean_ws:.3f} m/s")
print(f"the difference in percentage is: {change_percentage:.2f} %")

The number of NaN values in column U is 1521
The number of NaN values in column V is 1521
The number of NaN values in column Wind Speed is 1521
The number of NaN values in column Wind Direction is 1521

The month with the most NaN values in column U is September
The month with the most NaN values in column V is September
The month with the most NaN values in column Wind Speed is September
The month with the most NaN values in column Wind Direction is September

the orginal mean wind speed was: 5.299 m/s
the new mean wind speed is: 5.185 m/s
the difference in percentage is: -2.14 %
